In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

import pandas as pd
from collections import OrderedDict

import pprint
import re
import numpy as np
import matplotlib.pyplot as plt
import random

import plotly.express as px
px.defaults.template =  "plotly_white"

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
init_notebook_mode(connected=True)  
import glob
import uproot 
import ROOT
from tqdm import tqdm


import platform
print('Python:', platform.python_version())

Welcome to JupyROOT 6.26/06
Python: 3.10.8


In [2]:

filenames = glob.glob(f"/sphenix/lustre01/sphnxpro/commissioning/online_monitoring/histograms/TPCMON/Run_54[78]*-TPCMON_[269].root") 
filenames = filenames + glob.glob(f"/sphenix/lustre01/sphnxpro/commissioning/online_monitoring/histograms/TPCMON/Run_54[78]*-TPCMON_1[28].root") 
filenames = filenames + glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_*-TPCMON_[269].root")
filenames = filenames + glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_*-TPCMON_1[28].root")
# filenames = glob.glob(f"/sphenix/lustre01/sphnxpro/commissioning/online_monitoring/histograms/TPCMON/Run_54[78]*-TPCMON_[9].root") 
# filenames = filenames + glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_*-TPCMON_[9].root")
filenames.sort()
pprint.pprint(filenames)

['/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54873-TPCMON_12.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54873-TPCMON_18.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54873-TPCMON_2.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54873-TPCMON_6.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54873-TPCMON_9.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54896-TPCMON_12.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54896-TPCMON_18.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54896-TPCMON_2.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54896-TPCMON_6.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/Run24/online_monitoring/histograms/Run_54896-TP

In [3]:
%%cpp -d

#include<sPhenixStyle.C>

In [4]:
%%cpp

SetsPhenixStyle()

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [5]:
%jsroot on

In [6]:

pedestal_sample_time = [20, 350]
drift_sample_time = [100, 250]
sample_width = 5


expo = ROOT.TF1("expo_pedestal", "[0] + [1]*exp([2]*x)", 60, 1000)

In [17]:
file_tmp = ROOT.TFile.Open(filenames[50])


c1 = ROOT.TCanvas( 'QA', 'QA', 1000, 500)

hist = file_tmp.Get("COUNTS_vs_SAMPLE_1D_R3")
expo.SetParameters(hist.GetBinContent(2), hist.GetMaximum(), -0.01)
hist.Fit("expo_pedestal", "RMS0", "", 60, 300)
hist.DrawClone()
expo.DrawClone("same")
print(f"decay constant: {-1/expo.GetParameter(2)}")
c1.Draw("")



decay constant: 147.37113983323994
 FCN=1245.57 FROM HESSE     STATUS=OK             18 CALLS         160 TOTAL
                     EDM=2.62308e-11    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -4.17039e+03   1.45648e+02   2.32847e-01   6.41458e-08
   2  p1           1.84939e+05   1.26658e+02   8.05236e-01   3.15209e-08
   3  p2          -6.78559e-03   1.83017e-05   2.64044e-08   2.36378e-01


Warning in <TCanvas::Constructor>: Deleting canvas with same name: QA


In [ ]:

runnumber_list = []
sector_list = []
decay_constant_list = []
decay_constant_err_list = []

first_run = True

for filename in tqdm(filenames):
    print(f"Openning filename ... {filename}")
    file = ROOT.TFile.Open(filename)

    runnumber = int(filename.split("-")[-2].split("_")[-1])
    sector = int(filename.split(".")[-2].split("_")[-1])

    list = file.GetListOfKeys()

    first_hist = True

    for key in list:
        keyname = key.GetName()
        if not re.match("COUNTS_vs_SAMPLE_1D_R3", keyname):
            continue

        hist = file.Get(keyname)
        if first_hist:
            hist_sum = hist.Clone(f"h_{runnumber}_{sector}")
            first_hist = False
        else:
            hist_sum.Add(hist)
        
        print(f"{runnumber}/Sector{sector}/{keyname}:  statistics {hist_sum.GetSum()}")
    
    if hist_sum.GetSum() < 1000000:
        print(f"Skipping run {runnumber}/Sector{sector} due to low statistics {hist_sum.GetSum()}")
        continue

    if not first_hist:
        expo.SetParameters(hist_sum.GetBinContent(2), hist_sum.GetMaximum(), -0.01)
        hist_sum.Fit("expo_pedestal", "RMS0", "", 60, 300)
        decay_constant = -1/expo.GetParameter(2)
        if abs(expo.GetParError(2)/expo.GetParameter(2)) > 0.1:
            print(f"Skipping run {runnumber}/Sector{sector} due to large error {abs(expo.GetParError(2)/expo.GetParameter(2))}")
            continue

        runnumber_list.append(runnumber)
        sector_list.append(sector)
        decay_constant_list.append( decay_constant )
        decay_constant_err_list.append(abs(expo.GetParError(2)/expo.GetParameter(2)*decay_constant))

        if first_run:
            first_run = False

    file.Close()




In [ ]:
# pprint.pprint(decay_constant_dict)
df = pd.DataFrame({'Run Number': runnumber_list, 'Sector': sector_list, 
                   'Decay Constant [50ns]': decay_constant_list,
                   'Decay Constant Error': decay_constant_err_list})
print (df)

In [11]:

fig = px.scatter(df, x="Run Number", y="Decay Constant [50ns]", error_y="Decay Constant Error", 
                 color="Sector", labels={"Sector": "Sector"},
                 title="sPHENIX Internal, Run24 AuAu, TPC [ADC Hit vs drift time]'s Exponential Decay Constant"
                 , height=800)
fig.update_xaxes(tickformat="d")
fig.update_layout(yaxis=dict(range=[0, 300]))

fig.show()